In [2]:
import numpy as np
import pandas as pd
from mp_api.client import MPRester
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from matminer.featurizers.structure import DensityFeatures
from matminer.featurizers.structure import StructuralComplexity
from matminer.featurizers.structure import MaximumPackingEfficiency

C:\Users\Alexander\anaconda3\envs\materials\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing necessary packages: <br>
Numpy and Pandas for data handling and manipulation. <br>

Scikit learn's Random Forest Classifier train-test-split for building our stability-classifying model and evaluating its performance. <br>
- A Random Forest was chosen for its high predictive power when used in smaller datasets; the number of real perovskites stored on the Materials Project was <2000, thus a Random Forest was opted for (and performed the best). A classifier was chosen as we are initially only predicting whether the material is stable or not.

Three of matminer's featurizer packages, all based on the material's structure.
- Various combinations of different featurizers provided by matminer were tested together, and the following three provided the highest accuracy, for reasons later explained.

Density features
- Includes Density, VPA (the average volume occupied by each atom in a material), and Packing Fraction (the fraction of space occupied by particles in a given volume).

Structural complexity
- Features relating to Shannon Information Entropy (used to assess variability in the arrangement of atoms), includes Structural Complexity per Atom / per Cell.

Maximum packing efficiency
- Another method for assessing the arrangement of atoms in a material, includes only one value.

### Perovskites

In [2]:
with MPRester("EclqWMXn0DY3CAOSELD3xvCjEVhlcEYp") as mpr:
    perovskites = mpr.summary.search(formula=["ABC3"], fields = ["material_id", "structure", "band_gap", "theoretical", "is_stable", "formula_pretty", "composition"])

C:\Users\Alexander\AppData\Local\Temp\ipykernel_9340\4088062957.py:2: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  perovskites = mpr.summary.search(formula=["ABC3"], fields = ["material_id", "structure", "band_gap", "theoretical", "is_stable", "formula_pretty", "composition"])
Retrieving SummaryDoc documents: 100%|██████████| 4555/4555 [00:07<00:00, 623.81it/s]


Using the Materials Project's new API for querying data; collecting information for all materials with general formula 'ABC3' (perovskites) including the material's ID, structure, band gap, formula, composition, stability, and whether or not it has been sythnesised.

In [5]:
ids = []
for i in range(len(perovskites)):
    single = perovskites[i]
    ids.append(single.material_id)

structures = []
for i in range(len(perovskites)):
    single = perovskites[i]
    structures.append(single.structure)

band_gaps = []
for i in range(len(perovskites)):
    single = perovskites[i]
    band_gaps.append(single.band_gap)

theory = []
for i in range(len(perovskites)):
    single = perovskites[i]
    theory.append(single.theoretical)

stable = []
for i in range(len(perovskites)):
    single = perovskites[i]
    stable.append(single.is_stable)

formula = []
for i in range(len(perovskites)):
    single = perovskites[i]
    formula.append(single.formula_pretty)

composition = []
for i in range(len(perovskites)):
    single = perovskites[i]
    composition.append(single.composition)

perov_df = pd.DataFrame(
    {"material ids": ids,
     "formula": formula,
     "composition": composition,
     "structure": structures,
     "band gaps": band_gaps,
     "theoretical": theory,
     "stable": stable
    })

Creating a dataframe containing all information gathered on perovskites for ease in handling.

In [ ]:
real_perovs = perov_df[perov_df["theoretical"]==False]

Removing all hypothetical materials from our dataset

In [8]:
real_perovs["stable"] = real_perovs["stable"].astype(int)

C:\Users\Alexander\AppData\Local\Temp\ipykernel_9340\3917787968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_perovs["stable"] = real_perovs["stable"].astype(int)


Converting the values in the 'stable' from True/False to 1/0 

In [9]:
perovs_feat = real_perovs

In [7]:
densityf = DensityFeatures()
strcomp = StructuralComplexity()
mpe = MaximumPackingEfficiency()

Storing the structure featurizing functions as simpler objects

In [12]:
perovs_feat = densityf.featurize_dataframe(perovs_feat, "structure")

DensityFeatures: 100%|██████████| 1759/1759 [04:42<00:00,  6.22it/s]


Generating density features for the perovskite dataset based on the structures.

In [13]:
perovs_feat = strcomp.featurize_dataframe(perovs_feat, "structure")

StructuralComplexity: 100%|██████████| 1759/1759 [04:49<00:00,  6.08it/s]


Generating features of structural complexity for the perovskite dataset based on the structures.

In [14]:
perovs_feat = mpe.featurize_dataframe(perovs_feat, "structure")

MaximumPackingEfficiency: 100%|██████████| 1759/1759 [11:45<00:00,  2.49it/s]


Generating features of maximum packing efficiency for the perovskite dataset based on the structures.

In [15]:
y = perovs_feat["stable"].values
X = perovs_feat.drop(["material ids", "formula", "composition", "structure", "band gaps", "theoretical", "stable"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

stable_classifier = RandomForestClassifier(random_state=0)
stable_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

Fitting a Random Forest Classifer to predict perovskite stability based on the information generated from density features, structural complexity, and maximum packing efficiency.

In [16]:
training_accuracy = stable_classifier.score(X_train, y_train) * 100
print(f'Accuracy on training data: {training_accuracy:.1f} %')
testing_accuracy = stable_classifier.score(X_test, y_test) * 100
print(f'Accuracy on testing data: {testing_accuracy:.1f} %')

Accuracy on training data: 100.0 %
Accuracy on testing data: 71.1 %


Testing the perovskite-stability classifier.

### III-V Semiconductors

In [17]:
with MPRester("EclqWMXn0DY3CAOSELD3xvCjEVhlcEYp") as mpr:
    iii_v = mpr.summary.search(chemsys=["Al-N", "Al-P", "Al-As", "Al-Sb",
                                      "Ga-N", "Ga-P", "Ga-As", "Ga-Sb"
                                      "In-N", "In-P", "In-As", "In-Sb"], 
                              fields = ["material_id", "structure", "band_gap", "theoretical", "is_stable", "formula_pretty", "composition"])

C:\Users\Alexander\AppData\Local\Temp\ipykernel_9340\2391299554.py:2: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  iii_v = mpr.summary.search(chemsys=["Al-N", "Al-P", "Al-As", "Al-Sb",
Retrieving SummaryDoc documents: 100%|██████████| 117/117 [00:00<?, ?it/s]


Using the Materials Project's new API for querying data; collecting information for all III-V semiconductors, including the material's ID, structure, band gap, formula, composition, stability, and whether or not it has been sythnesised.

In [19]:
ids = []
for i in range(len(iii_v)):
    single = iii_v[i]
    ids.append(single.material_id)

structures = []
for i in range(len(iii_v)):
    single = iii_v[i]
    structures.append(single.structure)

band_gaps = []
for i in range(len(iii_v)):
    single = iii_v[i]
    band_gaps.append(single.band_gap)

theory = []
for i in range(len(iii_v)):
    single = iii_v[i]
    theory.append(single.theoretical)

stable = []
for i in range(len(iii_v)):
    single = iii_v[i]
    stable.append(single.is_stable)

formula = []
for i in range(len(iii_v)):
    single = iii_v[i]
    formula.append(single.formula_pretty)

composition = []
for i in range(len(iii_v)):
    single = iii_v[i]
    composition.append(single.composition)

iii_v_df = pd.DataFrame(
    {"material ids": ids,
     "formula": formula,
     "composition": composition,
     "structure": structures,
     "band gaps": band_gaps,
     "theoretical": theory,
     "stable": stable
    })

Creating a dataframe containing all information gathered on III-V semiconductor for ease in handling.

In [20]:
iii_v_feat = iii_v_df

In [21]:
iii_v_feat = densityf.featurize_dataframe(iii_v_feat, "structure")

DensityFeatures: 100%|██████████| 117/117 [00:10<00:00, 10.75it/s]


Generating density features for the III-V semiconductor dataset based on the structures.

In [22]:
iii_v_feat = strcomp.featurize_dataframe(iii_v_feat, "structure")

StructuralComplexity: 100%|██████████| 117/117 [00:13<00:00,  8.51it/s]


Generating features of structural complexity for the III-V semiconductor dataset based on the structures.

In [23]:
iii_v_feat = mpe.featurize_dataframe(iii_v_feat, "structure")

MaximumPackingEfficiency: 100%|██████████| 117/117 [00:16<00:00,  7.14it/s]


Generating features of maximum packing efficiency for the III-V semiconductor dataset based on the structures.

In [24]:
y = iii_v_feat["stable"].values
X = iii_v_feat.drop(["material ids", "formula", "composition", "structure", "band gaps", "theoretical", "stable"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

stable_classifier = RandomForestClassifier(random_state=0)
stable_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

Fitting a Random Forest Classifer to predict III-V semiconductor stability based on the information generated from density features, structural complexity, and maximum packing efficiency.

In [25]:
training_accuracy = stable_classifier.score(X_train, y_train) * 100
print(f'Accuracy on training data: {training_accuracy:.1f} %')
testing_accuracy = stable_classifier.score(X_test, y_test) * 100
print(f'Accuracy on testing data: {testing_accuracy:.1f} %')

Accuracy on training data: 100.0 %
Accuracy on testing data: 96.7 %


Testing the III-V semiconductor-stability classifier.

### II-VI Semiconductors

In [3]:
with MPRester("EclqWMXn0DY3CAOSELD3xvCjEVhlcEYp") as mpr:
    ii_vi = mpr.summary.search(chemsys=["Zn-S", "Zn-Se", "Zn-Te",
                                        "Cd-S", "Cd-Se", "Cd-Te"], 
                               fields = ["material_id", "structure", "band_gap", "theoretical", "is_stable", "formula_pretty", "composition"])

C:\Users\Alexander\AppData\Local\Temp\ipykernel_10064\1361324371.py:2: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  ii_vi = mpr.summary.search(chemsys=["Zn-S", "Zn-Se", "Zn-Te",
Retrieving SummaryDoc documents: 100%|██████████| 185/185 [00:00<?, ?it/s]


Using the Materials Project's new API for querying data; collecting information for all II-VI semiconductors, including the material's ID, structure, band gap, formula, composition, stability, and whether or not it has been sythnesised.

In [4]:
ids = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    ids.append(single.material_id)

structures = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    structures.append(single.structure)

band_gaps = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    band_gaps.append(single.band_gap)

theory = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    theory.append(single.theoretical)

stable = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    stable.append(single.is_stable)

formula = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    formula.append(single.formula_pretty)

composition = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    composition.append(single.composition)

ii_vi_df = pd.DataFrame(
    {"material ids": ids,
     "formula": formula,
     "composition": composition,
     "structure": structures,
     "band gaps": band_gaps,
     "theoretical": theory,
     "stable": stable
    })

Creating a dataframe containing all information gathered on II-VI semiconductors for ease in handling.

In [5]:
ii_vi_feat = ii_vi_df

In [8]:
ii_vi_feat = densityf.featurize_dataframe(ii_vi_feat, "structure")

DensityFeatures: 100%|██████████| 185/185 [00:41<00:00,  4.43it/s]


Generating density features for the II-VI semiconductor dataset based on the structures.

In [9]:
ii_vi_feat = strcomp.featurize_dataframe(ii_vi_feat, "structure")

StructuralComplexity: 100%|██████████| 185/185 [00:37<00:00,  4.91it/s]


Generating features of structural complexity for the II-VI semiconductor dataset based on the structures.

In [11]:
ii_vi_feat = mpe.featurize_dataframe(ii_vi_feat, "structure")

MaximumPackingEfficiency: 100%|██████████| 185/185 [01:57<00:00,  1.58it/s]


Generating features of maximum packing efficiency for the II-VI semiconductor dataset based on the structures.

In [32]:
y = ii_vi_feat["stable"].values
X = ii_vi_feat.drop(["material ids", "formula", "composition", "structure", "band gaps", "theoretical", "stable"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

stable_classifier = RandomForestClassifier(random_state=0)
stable_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

Fitting a Random Forest Classifer to predict II-VI semiconductor stability based on the information generated from density features, structural complexity, and maximum packing efficiency.

In [33]:
training_accuracy = stable_classifier.score(X_train, y_train) * 100
print(f'Accuracy on training data: {training_accuracy:.1f} %')
testing_accuracy = stable_classifier.score(X_test, y_test) * 100
print(f'Accuracy on testing data: {testing_accuracy:.1f} %')

Accuracy on training data: 100.0 %
Accuracy on testing data: 95.7 %


Testing the II-VI semiconductor-stability classifier.